In [0]:

from sodapy import Socrata

#client = Socrata("www.datos.gov.co", None)

token = dbutils.secrets.get("claves","token_app")
codigo_dataset = dbutils.widgets.get("codigo_dataset")

client = Socrata("www.datos.gov.co",str(token),timeout=1000)

In [0]:
from sodapy import Socrata
import time

# Datos conocidos
TOTAL_REGISTROS = 19450865
limit = 500000
offset = 0
write_mode = "overwrite"
reintentos = 5

# Conexión
codigo_dataset = dbutils.widgets.get("codigo_dataset")
token = dbutils.secrets.get("claves", "token_app")
client = Socrata("www.datos.gov.co", str(token), timeout=1000)

print(f"🚀 Iniciando carga de {TOTAL_REGISTROS:,} registros del dataset: {codigo_dataset}")

while offset < TOTAL_REGISTROS:
    intentos = 0

    while intentos < reintentos:
        try:
            print(f"🔄 Obteniendo y cargando lote desde offset {offset}...")

            query = f"""
                SELECT numero_del_contrato, numero_de_proceso, nit_de_la_entidad,
                       documento_proveedor, estado_del_proceso
                LIMIT {limit} OFFSET {offset}
            """
            results = client.get(codigo_dataset, query=query)

            if not results:
                print("✅ Lote vacío recibido. Fin de la carga.")
                offset = TOTAL_REGISTROS  # Forzar salida del bucle principal
                break

            # Convertir a Spark DataFrame y guardar en tabla Delta
            spark.createDataFrame(results).write \
                .format("delta") \
                .mode(write_mode) \
                .option("overwriteSchema", "true") \
                .saveAsTable("main.diplomado_datos.ids_contratos_procesos")

            print(f"📦 Lote de {len(results):,} registros desde offset {offset} cargado.")
            offset += limit
            write_mode = "append"
            break

        except Exception as e:
            intentos += 1
            print(f"🚨 Error al obtener o cargar lote: {e}")
            print(f"🔁 Reintentando ({intentos}/{reintentos})...")
            time.sleep(40)

    else:
        print("❌ Se alcanzó el número máximo de intentos. Cancelando proceso.")
        break

print("🏁 Proceso finalizado correctamente.")

